# Материалы

https://glom.readthedocs.io/

In [1]:
from glom import glom, Coalesce, T, Iter, Merge, Val

In [2]:
import sys
sys.version

'3.10.4 (main, Jun 27 2022, 16:57:57) [GCC 11.2.0]'

## Accessing Nested Data ("deep access")

In [3]:
data = {
    'a': {
        'b': {
            'c': 10
        }
    }
}
glom(data, 'a.b.c'), glom(data, T['a']['b']['c'])

(10, 10)

In [4]:
try:
    glom(data, 'a.b.d')
except Exception as e:
    print(e)

error raised while processing, details below.
 Target-spec trace (most recent last):
 - Target: {'a': {'b': {'c': 10}}}
 - Spec: 'a.b.d'
glom.core.PathAccessError: could not access 'd', part 2 of Path('a', 'b', 'd'), got error: KeyError('d')


## Going Beyond Access

`target` is our data, be it a dict, list, or any other object

`spec` is what we want output to be

output = glom(target, spec)

In [5]:
target = {
    'system': {
        'planets': [
            {
                'name': 'earth',
                'moons': [
                    {'name': 'luna'}
                ]
            },
            {
                'name': 'jupiter',
                'moons': [
                    {'name': 'io'},
                    {'name': 'europa'}
                ]
            }
        ]
    }
}

In [6]:
spec = {
    'planet_names': ('system.planets', ['name']),
    'moon_names': ('system.planets', [('moons', ['name'])])
}
glom(target, spec)

{'planet_names': ['earth', 'jupiter'],
 'moon_names': [['luna'], ['io', 'europa']]}

## Data-Driven Assignment

Quite often APIs deliver data in dictionaries without constant key values. They use parts of the data itself as a key. This we call `data-driven` assignment.

In [7]:
target = {
   'pluto': {'moons': 6, 'population': None},
   'venus': {'population': {'aliens': 5}},
   'earth': {'moons': 1, 'population': {'humans': 7700000000, 'aliens': 1}},
}

spec = {
    'moons': (
         T.items(),
         Iter({T[0]: (T[1], Coalesce('moons', default=0))}),
         Merge(),
    )
}
glom(target, spec)

{'moons': {'pluto': 6, 'venus': 0, 'earth': 1}}

## True Python Native

Most other implementations are limited to a particular data format or pure model, be it `jmespath` or `XPath/XSLT`. `glom` makes no such sacrifices of practicality, harnessing the full power of Python itself.

With `glom`, you have full access to Python at any given moment. Pass values to functions, whether built-in, imported, or defined inline with lambda.

In [8]:
target = {
    'system': {
        'planets': [
            {'name': 'earth', 'moons': 1},
            {'name': 'jupiter', 'moons': 69}
        ]
    }
}
glom(target, {'moon_count': ('system.planets', ['moons'], sum)})

{'moon_count': 70}

## Мои попытки

In [9]:
persons = [
    {
        'name': 'Igor',
        'age': 55,
        'children_count': 2
    },
    {
        'name': 'Mike',
        'age': 40,
        'children_count': 5
    },
    {
        'name': 'Olga',
        'age': 20
    },
]

In [10]:
spec = {
    'children_total_count': ([Coalesce('children_count', default=0)], sum),
    'some_val': Val('val'),
    'oldest employee': (['age'], max),
    'names': ['name']
}
glom(persons, spec)

{'children_total_count': 7,
 'some_val': 'val',
 'oldest employee': 55,
 'names': ['Igor', 'Mike', 'Olga']}

--------------------------------------

## Iter

In [11]:
glom(['10', '20', '30'], (Iter(T[0]).map(T + '-'), lambda x:''.join(x)))

'1-2-3-'